# Setup

In [ ]:
# Google Only
from google.colab import drive
drive.mount('/content/drive')

In [9]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath('')))
sys.path.append('/content/drive/My Drive/Code/autocomplete_me')

In [68]:
root_folder = os.path.dirname(os.path.abspath(''))

'/Users/jaipancholi/Code/autocomplete_me'

In [10]:
from src import utils, reader, predict_utils

Using TensorFlow backend.


## Load Text Data

In [11]:
text = reader.read_bbc_tech()
content_type = 'BBC-tech'

In [12]:
text[0]

'Mobiles rack up 20 years of use\n\nMobile phones in the UK are celebrating their 20th anniversary this weekend.\n\nBritain\'s first mobile phone call was made across the Vodafone network on 1 January 1985 by veteran comedian Ernie Wise. In the 20 years since that day, mobile phones have become an integral part of modern life and now almost 90% of Britons own a handset. Mobiles have become so popular that many people use their handset as their only phone and rarely use a landline.\n\nThe first ever call over a portable phone was made in 1973 in New York but it took 10 years for the first commercial mobile service to be launched. The UK was not far behind the rest of the world in setting up networks in 1985 that let people make calls while they walked. The first call was made from St Katherine\'s dock to Vodafone\'s head office in Newbury which at the time was over a curry house. For the first nine days of 1985 Vodafone was the only firm with a mobile network in the UK. Then on 10 Janua

# Modelling

In [80]:
import tensorflow as tf
import numpy as np

## Process Text Data

In [46]:
# Train Tokenizer and Apply
from tensorflow.keras.preprocessing.text import Tokenizer

# Train
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

# Apply Tokenzier on Documents (convert words to numbers)
sequences = tokenizer.texts_to_sequences(text)

num_words = len(tokenizer.word_index) + 1

In [52]:
# Create Sliding Window
sequence_len = 10

features = []
labels = []

for sequence in sequences:
    for i in range(len(sequence) - sequence_len):
        window = sequence[i:sequence_len + i + 1]
        
        features.append(window[:-1])
        labels.append(window[-1])

print(f'There are {len(features)} sequences.')

There are 200287 sequences.


In [53]:
# One Hot Encode Labels
from tensorflow.keras.utils import to_categorical
labels = to_categorical(labels)
print('Labels matrix shape: ', labels.shape)

Labels matrix shape:  (200287, 12676)


In [81]:
# Create Test Train Set
from sklearn.model_selection import train_test_split

features = np.array(features)
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.20, random_state=42, shuffle=True)

In [84]:
X_train

array([[  32,   21,   26, ...,  619,    3, 4156],
       [6698,   10,    8, ...,    1,   65,    9],
       [   1,  287,   13, ...,    1,  579,   53],
       ...,
       [2637,  204,   76, ...,  108,  145,   16],
       [   1, 1012, 4852, ...,  410,   46,   38],
       [   1,  111,    4, ...,   64,   47,  369]])

In [83]:
print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

X_train shape:  (160229, 10)
X_test shape:  (40058, 10)
y_train shape:  (160229, 12676)
y_test shape:  (40058, 12676)


In [72]:
# Embedding Matrix
# embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/Users/jaipancholi/data/glove.6B.100d.txt')
embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/content/drive/My Drive/Code/autocomplete_me/data/glove.6B.100d.txt')
embedding_matrix

Glove Vectors loading with dimension 100
There were 1062 words without pre-trained embeddings.


/Users/jaipancholi/Code/autocomplete_me/src/utils.py:180: RuntimeWarning: invalid value encountered in true_divide
  # embedding_matrix = embedding_matrix / np.linalg.norm(embedding_matrix, axis=1).reshape((-1, 1))


# Design Model

In [89]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense

tensorflow.python.keras.layers.recurrent_v2.LSTM

In [95]:
model = Sequential()

model.add(    
    Embedding(
    input_dim=num_words,
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    trainable=True)
)

model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))

# output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train Model

In [96]:
from tensorflow.keras.callbacks import ModelCheckpoint

ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath('')))
MODELS_DIR = os.path.join(ROOT_DIR, 'models')

model_filename = 'test.h5'
model_filepath = os.path.join(MODELS_DIR, model_filename)

callbacks = [
    ModelCheckpoint(f'{model_filepath}', save_best_only=True, save_weights_only=False)
]

EPOCHS = 100

history = model.fit(
    X_train, 
    y_train, 
    epochs=EPOCHS, 
    batch_size=2048, 
    validation_data=(X_test, y_test), 
    verbose=1,
    callbacks=callbacks
)

Train on 160229 samples, validate on 40058 samples
Epoch 1/100
 10240/160229 [>.............................] - ETA: 3:16 - loss: 9.4367 - accuracy: 0.0428WARNING:tensorflow:Can save best model only with val_loss available, skipping.


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/jaipancholi/Code/autocomplete_me/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-96-46953632504e>", line 22, in <module>
    callbacks=callbacks
  File "/Users/jaipancholi/Code/autocomplete_me/venv/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training.py", line 728, in fit
    use_multiprocessing=use_multiprocessing)
  File "/Users/jaipancholi/Code/autocomplete_me/venv/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_v2.py", line 324, in fit
    total_epochs=epochs)
  File "/Users/jaipancholi/Code/autocomplete_me/venv/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_v2.py", line 123, in run_one_epoch
    batch_outs = execution_function(iterator)
  File "/Users/jaipancholi/Code/autocomplete_me/venv/lib/python3.7/site-packages/tensorflow_core/python/keras

TypeError: object of type 'NoneType' has no len()